<a href="https://colab.research.google.com/github/arcossci/Finetuning-models/blob/main/Create_dataset_CoT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: install packages

!pip install openai PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00


In [19]:
import os
import glob
import openai
import PyPDF2
import json
import pandas as pd

# Set up OpenAI client
from google.colab import userdata
client = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

def extract_text_from_pdf(pdf_path):
    """Extracts all text from a PDF file."""
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

def split_text_into_sections(text, max_chars=1000):
    """Splits text into sections of up to max_chars characters."""
    sections = []
    while text:
        if len(text) > max_chars:
            section = text[:max_chars]
            last_newline = section.rfind("\n")
            if last_newline != -1:
                section = section[:last_newline]
            sections.append(section)
            text = text[len(section):]
        else:
            sections.append(text)
            break
    return sections

def extract_json_from_response(response):
    """Extracts the JSON content from an LLM response, removing code block markers."""
    start_marker = "```json"
    end_marker = "```"
    start_index = response.find(start_marker)
    if start_index != -1:
        start_index += len(start_marker)
        end_index = response.find(end_marker, start_index)
        if end_index != -1:
            return response[start_index:end_index].strip()
    return response.strip()

def generate_cot_data(seccion_texto):
    """Generates CoT data from a text section using the LLM."""
    prompt = f"""Se te proporciona un extracto de documentos normativos sobre construcción en Colombia ley 2172 de 2021
      Tu tarea es derivar al menos dos preguntas legales distintas y de alta calidad que un usuario (abogado, ciudadano, etc.) podría formular basándose en el contenido.
      Sé muy creativo anticipando preguntas que los usuarios suelen hacer.

      Para cada pregunta, proporciona:
      - Question: La pregunta legal formulada a partir del texto.
      - Complex_CoT: Dada la pregunta y respuesta, descompón el razonamiento paso a paso siguiendo el enfoque Chain-of-Thought. Explica cada paso lógico que llevó a la conclusión final.
      - Response: Una respuesta concisa basada en el extracto.

      Asegúrate de que las preguntas, el razonamiento y las respuestas sean precisos y útiles para usuarios reales.
      No solo uses que, sino aleatoriamente: 'quién, quiénes, cuál, cuáles, cómo, cuánto, cuánta, cuántos, cuántas, dónde, cuándo, por qué' apropiadamente
      debes inventar preguntas que sean coherentes con el texto.

      Extracto:
      {seccion_texto}

      Devuelve un array JSON válido con exactamente cinco objetos. Cada objeto debe contener únicamente las claves "Question", "Complex_CoT" y "Response".

      Salida:
      """
    respuesta = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Eres un asistente experto en normatividad legal colombiana."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=1000,
    )
    return respuesta.choices[0].message.content

def main():
    pdf_directory = "/content/drive/MyDrive/Startup ideas/LegalTech/base"
    pdf_paths = glob.glob(os.path.join(pdf_directory, "*.pdf"))

    data_records = []

    for pdf_path in pdf_paths:
        full_text = extract_text_from_pdf(pdf_path)
        sections = split_text_into_sections(full_text)
        for section in sections:
            generated_output = generate_cot_data(section)
            json_str = extract_json_from_response(generated_output)
            try:
                data_list = json.loads(json_str)
                if isinstance(data_list, list):
                    for item in data_list:
                        if all(key in item for key in ["Question", "Complex_CoT", "Response"]):
                            data_records.append(item)
                        else:
                            print("Skipping an item with missing keys:", item)
                else:
                    print("Output is not a list:", json_str)
            except Exception as e:
                print("Error parsing JSON:", e)
                print("Raw output:", generated_output)

    df = pd.DataFrame(data_records, columns=["Question", "Complex_CoT", "Response"])
    df.to_csv("cot_dataset.csv", index=False, encoding="utf-8")
    print("DataFrame created with shape:", df.shape)
    print(df.head())
    df.to_csv("/content/drive/MyDrive/Startup ideas/LegalTech/CoT_dataset/cot_dataset_2.csv", index=False, encoding="utf-8")

if __name__ == "__main__":
    main()

DataFrame created with shape: (27, 3)
                                            Question  \
0  ¿Cómo se determina quiénes son las beneficiari...   
1  ¿Cuáles son las implicaciones legales para las...   
2  ¿Cuáles son las implicaciones legales de la te...   
3  ¿Qué tipos de lesiones personales se considera...   
4  ¿Cómo se determina la calidad de víctima de vi...   

                                         Complex_CoT  \
0  1. Primero, se debe identificar el objetivo de...   
1  1. Se inicia analizando la Ley 2172 y su objet...   
2  Primero, se debe identificar que la tentativa ...   
3  Primero, se debe revisar el Capítulo 111 del T...   
4  Para responder a esta pregunta, primero es nec...   

                                            Response  
0  Las beneficiarias del subsidio de vivienda son...  
1  Las entidades que no garanticen el acceso a su...  
2  La tentativa de feminicidio puede tener serias...  
3  Las lesiones personales consideradas graves so...  
4  La calidad

In [18]:
df3 = pd.read_csv("/content/cot_dataset.csv")
df3.to_csv("/content/drive/MyDrive/Startup ideas/LegalTech/CoT_dataset/cot_dataset_1523_2012.csv", index=False, encoding="utf-8")